In [376]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
  


In [377]:
f = open("voters.txt")
txt = f.read()
# txt = re.sub(r'[{}]', '', txt)

In [378]:
preferences = [val.split(': ')[-1] for val in txt.split('\n')]
counts = [float(val.split(': ')[0]) for val in txt.split('\n')]

In [379]:
preferences_to_counts = {preference: count for preference, count in zip(preferences, counts)}
preferences_to_counts = defaultdict(int, preferences_to_counts)


new_preferece_counts = []
preferences_with_ties = []
for preference, count in zip(preferences, counts):
    ties = re.findall("{\d+,\d+}", preference)
    n_ties = len(ties)
    if(n_ties>0):
        print(preference)
        preferences_with_ties.append(preference)
#       we expect only one tie for each preference of alternatives
        tie = re.sub("[{}]", "", ties[0]).split(',')
        for t in tie:
            new_preferece_counts.append((re.sub(re.escape(ties[0]), t, preference), count/2))
            
for preference, extra_counts in new_preferece_counts:
    preferences_to_counts[preference] += extra_counts
for preference in preferences_with_ties:
    del preferences_to_counts[preference]
    

3,4,5,6,7,8,9,{1,10},2
2,4,5,7,{9,10},3,6,1,8
1,2,6,5,3,7,8,9,{10,11}
4,2,5,3,1,6,7,8,{9,10}
9,3,2,5,{1,4},6,7,8
2,8,3,4,{1,5},6,9,7
3,5,1,{7,9},8,6,4,2
3,1,2,5,6,8,{9,10}
2,4,1,{5,8},6,9


In [380]:
alternatives = [str(i) for i in range(1, 12)]
alternatives

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [381]:
preferences = [preference.split(',') for preference in list(preferences_to_counts.keys())]
counts = list(preferences_to_counts.values())

In [382]:
df =  pd.DataFrame([counts, preferences]).T
df.rename(columns={0: 'n', 1: 'preference'}, inplace=True)

In [383]:
# has +9 new preferences due to the ties introduced
df.tail(20)

,n,preference
1749,1.0,"[3, 6]"
1750,1.0,[9]
1751,0.5,"[3, 4, 5, 6, 7, 8, 9, 1, 2]"
1752,0.5,"[3, 4, 5, 6, 7, 8, 9, 10, 2]"
1753,0.5,"[2, 4, 5, 7, 9, 3, 6, 1, 8]"
1754,0.5,"[2, 4, 5, 7, 10, 3, 6, 1, 8]"
1755,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 10]"
1756,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 11]"
1757,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 9]"
1758,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 10]"


In [384]:
# number of total voters
df.n.sum()

2477.0

In [385]:
def get_p_scores(df, alternatives):
    scores = {}
    for a in alternatives:
        scores[a] = 0
    for index, row in df.iterrows():
        if(len(row.preference) > 0):
            scores[row.preference[0]] += int(row.n)
    return scores

def get_score_alternatives(scores, mode):
    mode_score = min(scores, key=scores.get)
    if(mode == 'max'):
        mode_score = max(scores, key=scores.get)
    mode_score_alternatives = [i for i, val in scores.items() if val == scores[mode_score]]
    return mode_score_alternatives

def remove_alternatives(df, alternatives):
    df.preference = df.apply(lambda x: list(set(x.preference).difference(set(alternatives))), axis=1)
    return df

def STV(df, alternatives):
    votes_df = df.copy()
    scores = get_p_scores(votes_df, alternatives)
    while(len(alternatives)>0):
        scores = get_p_scores(votes_df, alternatives)
        print(scores)
        min_p_alternatives = get_score_alternatives(scores, 'min')
        alternatives = list(set(alternatives).difference(set(min_p_alternatives)))
        if(len(alternatives)==0):
            print("Reached a decision")
            break
        print('removing alternatives', min_p_alternatives)
        df = remove_alternatives(votes_df, min_p_alternatives)
    return list(scores.keys())

In [386]:
STV(df, alternatives)

{'1': 246, '2': 453, '3': 412, '4': 384, '5': 352, '6': 27, '7': 94, '8': 463, '9': 37, '10': 0, '11': 0}
removing alternatives ['10', '11']
{'7': 1136, '8': 198, '3': 333, '1': 54, '5': 128, '2': 491, '6': 54, '9': 14, '4': 60}
removing alternatives ['9']
{'7': 1129, '8': 174, '3': 354, '1': 71, '5': 123, '2': 518, '6': 60, '4': 38}
removing alternatives ['4']
{'7': 1067, '8': 120, '3': 372, '1': 81, '5': 119, '2': 612, '6': 70}
removing alternatives ['6']
{'7': 1048, '8': 104, '3': 442, '1': 72, '5': 127, '2': 638}
removing alternatives ['1']
{'7': 916, '8': 105, '3': 509, '5': 127, '2': 754}
removing alternatives ['8']
{'2': 1531, '3': 509, '7': 139, '5': 127}
removing alternatives ['5']
{'2': 1531, '3': 509, '7': 139}
removing alternatives ['7']
{'2': 1531, '3': 529}
removing alternatives ['3']
{'2': 1531}
Reached a decision


['2']